<a href="https://colab.research.google.com/github/Thaleslsilva/DataScience/blob/master/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF-IDF Para Identificação das Palavras mais Relevantes em um Livro

TF-IDF significa "Frequência do Termo - Frequência Inversa de Documentos". 

Essa é uma técnica para quantificar uma palavra nos documentos; geralmente calculamos um peso para cada palavra, o que significa a importância da palavra no documento e no corpus. Este método é uma técnica amplamente usada em Recuperação de Informação e Mineração de Texto.

Se eu lhe der uma frase, por exemplo, "Este edifício é tão alto". É fácil para nós entender a sentença como conhecemos a semântica das palavras e da sentença. Mas como o computador entenderá essa frase? O computador pode entender qualquer dado apenas na forma de valor numérico. Portanto, por esse motivo, vetorizamos todo o texto para que o computador possa entender melhor o texto.

Ao vetorizar os documentos, podemos executar várias tarefas, como encontrar documentos relevantes, classificação, agrupamento e assim por diante. É a mesma coisa que acontece quando você realiza uma pesquisa no Google. As páginas da web são chamadas de documentos e o texto com o qual você pesquisa é chamado de consulta. o Google mantém uma representação fixa para todos os documentos. Quando você pesquisa com uma consulta, o Google encontra a relevância da consulta com todos os documentos, classifica-os na ordem em que é relevante e mostra os principais documentos. Todo esse processo é feito usando a forma vetorizada da consulta e dos documentos. Embora os algoritmos do Google sejam altamente sofisticados e otimizados, essa é a estrutura usada.

Vamos extrair um livro inteiro, construir nossas funções para TF e IDF e então identificar as palavras mais relevantes em algumas frases do livro.

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Imports
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Thales de Lima Silva" --iversions

### Preparando os Dados

https://www.gutenberg.org/files/158/158-h/158-h.htm

In [ ]:
# Carrega os dados
nltk.download('gutenberg')
dados_livro_emma = nltk.corpus.gutenberg.sents('austen-emma.txt')

In [ ]:
# Listas para receber as frases e as palavras do texto
dados_livro_emma_frases = []
dados_livro_emma_palavras = []

In [ ]:
# Loop para a tokenização
# "isalpha" retorna cada palavra dentro de uma mesma frase e então passa para a frase seguinte (https://docs.python.org/3/library/stdtypes.html)
nltk.download('punkt')
for sentence in dados_livro_emma:
    dados_livro_emma_frases.append([word.lower() for word in sentence if word.isalpha()])
    for word in sentence:
        if word.isalpha():
            dados_livro_emma_palavras.append(word.lower())

In [ ]:
# Vamos converter a lista de palavras em um conjunto (set)
dados_livro_emma_palavras = set(dados_livro_emma_palavras)

In [ ]:
# Visualiza as frases
dados_livro_emma_frases

In [ ]:
# Visualiza as palavras
dados_livro_emma_palavras

### Frequência do Termo

A Frequência do Termo mede a frequência de uma palavra em um documento. 

Isso depende muito do tamanho do documento e da generalidade da palavra, por exemplo, uma palavra muito comum como "era" pode aparecer várias vezes em um documento, mas se pegarmos dois documentos, um com 100 palavras e outro com 10.000, há uma alta probabilidade de que uma palavra comum como "era" possa estar mais presente no documento de 10.000 palavras. Mas não podemos dizer que o documento mais longo é mais importante que o documento mais curto. Por esse exato motivo, realizamos uma normalização no valor da frequência, dividindo a frequência com o número total de palavras no documento.

Lembre-se de que precisamos finalmente vetorizar o documento. Quando estamos planejando vetorizá-lo, não podemos considerar apenas as palavras que estão presentes nesse documento específico. Se fizermos isso, o comprimento do vetor será diferente para os dois documentos e não será possível calcular a semelhança. Então, o que fazemos é que vetorizar os documentos no vocabulário, que é a lista de todas as palavras possíveis no corpus.

Quando estamos vetorizando os documentos, verificamos a contagem de cada palavra. Na pior das hipóteses, se o termo não existir no documento, esse valor de TF específico será 0 e, em outro caso extremo, se todas as palavras no documento forem iguais, será 1. O valor final do documento normalizado estará no intervalo de [0 a 1], sendo 0, 1 inclusive.

**tf(t,d) = contagem de t em d / número de palavras em d**

Se já calculamos o valor do TF e se isso produz uma forma vetorizada do documento, por que não usar apenas o TF para encontrar a relevância entre os documentos? Por que precisamos da IDF?

Embora tenhamos calculado o valor do TF, ainda existem alguns problemas, por exemplo, palavras mais comuns como "é" terão valores muito altos, dando a essas palavras uma importância muito alta. Mas usar essas palavras para calcular a relevância produz maus resultados. 

Esse tipo de palavra comum é chamado de palavras de parada (stop words) e, embora removamos as stop words posteriormente na etapa de pré-processamento, descobrir a importância da palavra em todos os documentos e normalizando  esse valor, representa muito melhor os documentos.

In [ ]:
# Função para calcular a Termo Frequência
def TermFreq(documento, palavra):
    doc_length = len(documento)
    ocorrencias = len([w for w in documento if w == palavra])
    return ocorrencias / doc_length

In [ ]:
dados_livro_emma_frases[5]

In [ ]:
# Aplica a função
TermFreq(dados_livro_emma_frases[5], 'mother')

Podemos então criar um dicionário (vocabulário).

Cada palavra única será identificada de forma única no objeto de dicionário. Isso é necessário para criar representações de textos. O corpus Bag of Words é criado e será necessário para a construção do modelo TF-IDF. 

O dicionário é criado pela lista de palavras. As frases/documentos, etc., podem ser convertidos em uma lista de palavras e depois alimentados nos corpora como parâmetro.

In [ ]:
# Criamos um corpus Bag of words
def cria_dict():
    output = {}
    for word in dados_livro_emma_palavras:
        output[word] = 0
        for doc in dados_livro_emma_frases:
            if word in doc:
                output[word] += 1
    return output

In [ ]:
# Cria o dicionário
df_dict = cria_dict()

In [ ]:
# Filtra o dicionário
df_dict['mother']

### Frequência Inversa

Para compreender o que é IDF, primeiro temos que compreender o que é DF.

A **Frequência do Documento** (DF) mede a importância do documento em todo o corpus e é muito semelhante ao TF. A única diferença é que TF é contador de frequência para um termo t no documento d, onde DF é a contagem de ocorrências do termo t no conjunto de documentos N. 

Em outras palavras, DF é o número de documentos em que a palavra está presente. Consideramos uma ocorrência se o termo consistir no documento pelo menos uma vez, não precisamos saber o número de vezes que o termo está presente.

df (t) = ocorrência de t nos documentos

Para manter isso também em um intervalo, normalizamos dividindo com o número total de documentos. Nosso principal objetivo é conhecer a informatividade de um termo, e DF é o inverso exato dele. É por isso que invertemos o DF.

**Frequência Inversa de Documentos**

IDF é o inverso da frequência do documento que mede a informatividade do termo t. Quando calcularmos o IDF, será muito baixo para as palavras que mais ocorrem, como stop words (porque stop words como "é" estão presentes em quase todos os documentos e N / df atribuirá um valor muito baixo a essa palavra). Isso finalmente resulta o que queremos, uma ponderação relativa.

idf (t) = N / df

Agora, existem alguns outros problemas com o IDF, no caso de um corpus grande, digamos 10.000, o valor do IDF explode. 

Então, para diminuir o efeito, aplicamos o log ao IDF.

Durante o tempo de consulta, quando uma palavra que não está no vocabulário ocorre, o df será 0. Como não podemos dividir por 0, suavizamos o valor adicionando 1 ao denominador.

idf (t) = log (N / (df + 1))

Finalmente, considerando um valor multiplicativo de TF e IDF, obtemos a pontuação TF-IDF, existem muitas variações diferentes de TF-IDF, mas por enquanto vamos nos concentrar nessa versão básica.

**tf-idf (t, d) = tf (t, d) * log (N / (df + 1))**

In [ ]:
# Função para calcular a Frequência Inversa de Documentos
def InverseDocumentFrequency(word):
    N = len(dados_livro_emma_frases)
    try:
        df = df_dict[word] + 1
    except:
        df = 1
    return np.log(N/df)

In [ ]:
# Aplica a função
InverseDocumentFrequency('mother')

### TF/IDF

In [ ]:
# Função TF-IDF
def TFIDF(doc, word):
    tf = TermFreq(doc, word)
    idf = InverseDocumentFrequency(word)
    return tf * idf

In [ ]:
dados_livro_emma_frases[5]

In [ ]:
# Print
print('mother: ' + str(TFIDF(dados_livro_emma_frases[5], 'mother')))

In [ ]:
dados_livro_emma_frases[30]

In [ ]:
# Print
print('mother: ' + str(TFIDF(dados_livro_emma_frases[30], 'mother')))

# Fim